In [ ]:
import ee
import traceback
import sys,os
sys.path.append(os.path.dirname(os.path.abspath('__file__')))
from Func.Basic_tools import *
from Func.New_Correct import *
from Func.Correct_filter import *
from Func.S2_filter import *
from functools import partial
geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

In [ ]:
def CalDitor(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'],Origin_Method='RS',Method:str='1',
             AOI_buffer=None,Origin_scale=10,projScale=30): 
    
    volumetric_dict = my_slope_correction(s1_ascending,s1_descending,AOI_buffer,DEMCOPERNICUS,Model,Origin_scale,DistorMethed=Origin_Method)


    # -----------------------基于线性关系。检测几何畸变
    Templist_A = AuxiliaryLine2Point(volumetric_dict['ASCENDING'],volumetric_dict['ASCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['ASCENDING_parms']['coordinates_dict'],
                                volumetric_dict['ASCENDING_parms']['Auxiliarylines'],
                                projScale)

    Templist_D = AuxiliaryLine2Point(volumetric_dict['DESCENDING'],volumetric_dict['DESCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['DESCENDING_parms']['coordinates_dict'],
                                volumetric_dict['DESCENDING_parms']['Auxiliarylines'],
                                projScale)
    if   Method == '1':
        LeftLayoverA,RightLayoverA,ShadowA = Line_Correct(volumetric_dict['ASCENDING'],AOI_buffer,Templist_A,'ASCENDING',
                                                volumetric_dict['ASCENDING_parms']['proj'],projScale,Origin_scale,
                                                filt_distance=False, save_peak=False, line_points_connect=True,Peak_Llay=True,Peak_shdow=True,Peak_Rlay=True)
        LeftLayoverD,RightLayoverD,ShadowD =  Line_Correct(volumetric_dict['DESCENDING'],AOI_buffer,Templist_D,'DESCENDING',
                                                volumetric_dict['DESCENDING_parms']['proj'],projScale,Origin_scale,
                                                filt_distance=False,save_peak=False,line_points_connect=True,Peak_Llay=True,Peak_shdow=True,Peak_Rlay=True)
    elif Method == '2':
        LeftLayoverA,RightLayoverA,ShadowA = Line_Correct2(volumetric_dict['ASCENDING'],AOI_buffer,Templist_A,'ASCENDING',
                                                volumetric_dict['ASCENDING_parms']['proj'],projScale,Origin_scale)
        LeftLayoverD,RightLayoverD,ShadowD =  Line_Correct2(volumetric_dict['DESCENDING'],AOI_buffer,Templist_D,'DESCENDING',
                                                volumetric_dict['DESCENDING_parms']['proj'],projScale,Origin_scale)
    else:
        print('Method = {}'.format(Method))
        raise Exception('Method方法目前只有两种，请重新输入')
    # 如果不加这一段，constant会判断出错
    LeftLayoverA=LeftLayoverA.rename('LeftLayoverA');RightLayoverA=RightLayoverA.rename('RightLayoverA');ShadowA=ShadowA.rename('ShadowA')
    LeftLayoverD=LeftLayoverD.rename('LeftLayoverD');RightLayoverD=RightLayoverD.rename('RightLayoverD');ShadowD=ShadowD.rename('ShadowD')
    return LeftLayoverA,RightLayoverA,ShadowA,Templist_A,LeftLayoverD,RightLayoverD,ShadowD,Templist_D,volumetric_dict

In [ ]:
Southest_doom = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/ChinaShp/SouthestRegion')
#-------------------DEM选择
DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)
models = ['volume', 'surface', None]     # 地形矫正模型
Model = models[0]

Southest_doom_fishnet = geemap.fishnet(Southest_doom.first().geometry(), rows=120, cols=180, delta=1)
lenfish_net = Southest_doom_fishnet.size().getInfo()
Southest_doom_fishnet = Southest_doom_fishnet.toList(lenfish_net)
os.chdir('/root/GEE/几何畸变COPER')

In [ ]:
Wrong_dataIndex = []
for i in ([
 836, 5582, 6606]):
    AOI_buffer = ee.Feature(Southest_doom_fishnet.get(i)).geometry()
    AOI_buffer_100 = AOI_buffer.buffer(100)
    AOI_buffer_200 = AOI_buffer_100.buffer(100)
    
    s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD").filter(ee.Filter.eq('instrumentMode', 'IW')).filterBounds(AOI_buffer_200))
    s1_col = s1_col.map(partial(rm_nodata, AOI=AOI_buffer_200))
    s1_col = s1_col.filter(ee.Filter.eq('numNodata', 0))
    # s1_descending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).first()
    # s1_ascending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first()
    
    s1_ascending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    list_images = s1_ascending.toList(s1_ascending.size())
    s1_ascending = ee.Image(list_images.get(1))

    s1_descending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    list_images = s1_descending.toList(s1_descending.size())
    s1_descending = ee.Image(list_images.get(1))
    
    try:
        LeftLayoverA,RightLayoverA,ShadowA,Templist_A,LeftLayoverD,RightLayoverD,ShadowD,Templist_D,volumetric_dict = \
                CalDitor(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'],
                         Origin_Method='RS',Method='1',AOI_buffer=AOI_buffer,Origin_scale=10,projScale=30)
        DistorA = LeftLayoverA.rename('LeftLayoverA').addBands(RightLayoverA.rename('RightLayoverA')).addBands(ShadowA.rename('ShadowA'))
        if DistorA.bandNames().length().getInfo() != 0:
            Geemap_export(fileDirname=f'{i:04d}'+'_'+'DistortionA'+'.tif',image=DistorA,region=AOI_buffer,scale=10)
        DistorD = LeftLayoverD.rename('LeftLayoverD').addBands(RightLayoverD.rename('RightLayoverD')).addBands(ShadowD.rename('ShadowD'))
        if DistorD.bandNames().length().getInfo() != 0:
            Geemap_export(fileDirname=f'{i:04d}'+'_'+'DistortionD'+'.tif',image=DistorD,region=AOI_buffer,scale=10) 
    except:
        Wrong_dataIndex.append(i)
        # 记录错误信息
        with open('log.txt', 'a') as f: 
            f.write('Wrong index = {}\n'.format(i))
            f.write(traceback.format_exc())
            f.write('\n')
        print('错误已记录到log.txt')

# 绘图展示

In [ ]:
Map = geemap.Map()
Map.centerObject(Southest_doom_fishnet, zoom=15)
Map.addLayer(Southest_doom_fishnet, {}, 'Fishnet 2')
Map.addLayer(s1_col.first(), {}, 'TEST')
Map